In [ ]:
from openmmtools.testsystems import WaterBox
import matplotlib.pyplot as plt
from simtk import unit
import numpy as np
import torchani
import torch
import time
from IPython.core.display import display, HTML

# openmm units
distance_unit = unit.nanometer

# ANI-1 units and conversion factors
ani_distance_unit = unit.angstrom

nm_to_angstroms = (1.0 * distance_unit) / (1.0 * ani_distance_unit)

# hartree to kJ/mol conversion factor
hartree_to_kJ_mol = 2625.499638

nr_of_atoms = []
times_with_pbc = []
times_without_pbc = []
platform = 'cpu'

# testing different edge length
species = None
coordinates = None
device = None
model = None

for box_length in [15]:
    
    torchani.nn.torch.cuda.empty_cache()
    device = torch.device(platform)
    model = torchani.models.ANI1ccx()
    model = model.to(device)

    # generate waterbox
    testsystem = WaterBox(box_edge=box_length*unit.angstrom)
    x = testsystem.positions
    dhfr_elements = [atom.element.symbol for atom in testsystem.topology.atoms()]
    print('Nr of atoms in the system: {}'.format(len(dhfr_elements)))
    element_string = ''.join(dhfr_elements)
    
    nr_of_atoms.append(len(dhfr_elements))

    # species and coordinates to torch
    species = model.species_to_tensor(element_string).to(device).unsqueeze(0)

    coordinates = torch.tensor([x.value_in_unit(unit.nanometer)],
                            requires_grad=True, device=device, dtype=torch.float32)

    
    print(testsystem.system.getDefaultPeriodicBoxVectors())
    # define cell vector
    cell_vectors = torch.tensor([[box_length, 0.0, 0.0], [0.0, box_length, 0.0], [0.0, 0.0, box_length]], dtype=torch.float32)
    # define dimensions to update
    cell = torch.tensor([True, True, True], dtype=torch.bool)

      
    
    #############################
    # energy & force calculation with pbc
    #############################
    t0 = time.time()
    # calculate energy
    _, energy_in_hartree = model((species, coordinates * nm_to_angstroms, cell_vectors, cell))
    energy_in_kJ_mol = energy_in_hartree * hartree_to_kJ_mol
    print('Energy: {} kJ/mol'.format(energy_in_kJ_mol[0]))
    # calculate force
    derivative = torch.autograd.grad((energy_in_kJ_mol).sum(), coordinates)[0]
    F = - np.array(derivative.cpu())[0] 
    
    # get time
    t1 = time.time()
    t = t1-t0
    print('time: {}'.format(t))   
    times_with_pbc.append(t)

# plot results
plt.plot(nr_of_atoms, times_without_pbc, label='Energy evaluation without pbc')
plt.plot(nr_of_atoms, times_with_pbc, label='Energy evaluation with pbc')
plt.title('Nr of atoms vs time')
plt.xlabel('Nr of atoms')
plt.ylabel('Time [sec]')
plt.legend()
plt.show()


In [1]:
from openmmtools.testsystems import WaterBox
import matplotlib.pyplot as plt
from simtk import unit
import numpy as np
import torchani
import torch
import time
from IPython.core.display import display, HTML
import neutromeratio
from openmmtools.constants import kB
from simtk import unit
import numpy as np
import pickle
import mdtraj as md
import torchani
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
exp_results = pickle.load(open('../data/exp_results.pickle', 'rb'))

# specify the system you want to simulate
name = 'molDWRow_298'
#name = 'molDWRow_37'
#name = 'molDWRow_45'
#name = 'molDWRow_160'
#name = 'molDWRow_590'


from_mol_tautomer_idx = 1
to_mol_tautomer_idx = 2


t1_smiles = exp_results[name]['t1-smiles']
t2_smiles = exp_results[name]['t2-smiles']

# generate both rdkit mol
mols = { 't1' : neutromeratio.generate_rdkit_mol(t1_smiles), 't2' : neutromeratio.generate_rdkit_mol(t2_smiles) }
from_mol = mols[f"t{from_mol_tautomer_idx}"]
to_mol = mols[f"t{to_mol_tautomer_idx}"]
ani_input = neutromeratio.from_mol_to_ani_input(from_mol)

neutromeratio.add_solvent(pdb_filepath=f"../data/structures/{name}/{name}_tautomer_{from_mol_tautomer_idx}.pdb",
                          ani_input=ani_input,
                         pdb_output_filepath=f"../data/structures/{name}/{name}_tautomer_{from_mol_tautomer_idx}_solvated.pdb")

# extract hydrogen donor idx and hydrogen idx for from_mol
platform = 'cpu'
device = torch.device(platform)

model = neutromeratio.ani.LinearAlchemicalANI(alchemical_atom = -1, ani_input=ani_input, device = device)
model = model.to(device)
torch.set_num_threads(1)

# perform initial sampling

energy_function = neutromeratio.ANI1_force_and_energy(device = device,
                                          model = model,
                                          atom_list = ani_input['ligand_atoms'] + ani_input['solvent_atoms'],
                                          platform = platform,
                                          tautomer_transformation = {},
                                          )

langevin = neutromeratio.LangevinDynamics(atom_list = ani_input['ligand_atoms'] + ani_input['solvent_atoms'],
                            temperature = 300*unit.kelvin,
                            force = energy_function)





In [3]:


a = neutromeratio.utils.MonteCarloBarostat(20.0 * unit.angstrom, energy_function)
a.current_volumn
a.update_volumn(10)


False


AssertionError: 

In [ ]:
plt.plot(nr_of_atoms, times_without_pbc, label='Energy evaluation without pbc')
plt.plot(nr_of_atoms, times_with_pbc, label='Energy evaluation with pbc')
plt.title('Nr of atoms vs time')
plt.xlabel('Nr of atoms')
plt.ylabel('Time [sec]')
plt.legend()
plt.show()
